In [27]:
import logging
import json
from datetime import datetime
import pandas as pd
import numpy as np
from trading_ig.rest import IGService
from trading_ig.config import config
# import yfinance as yf
import matplotlib.pyplot as plt
from autoIG.utils import market_series


In [28]:
## Instatiate IG Service
ig_service = IGService(config.username, config.password, config.api_key)
ig = ig_service.create_session()
# ig

In [29]:
## Search for a market
results = ig_service.search_markets('crypto')
results.head(2)

,epic,instrumentName,instrumentType,expiry,high,low,percentageChange,netChange,updateTime,updateTimeUTC,bid,offer,delayTime,streamingPricesAvailable,marketStatus,scalingFactor
0,CS.D.CRYPTOB10.TODAY.IP,Crypto 10 Index,CURRENCIES,DFB,6156.1,5985.2,0.18,10.73,14:10:42,14:10:42,6009.59,6129.59,0,True,TRADEABLE,1
1,CS.D.BITCOIN.TODAY.IP,Bitcoin,CURRENCIES,DFB,17243.2,17058.1,0.08,13.10,14:10:43,14:10:43,17104.00,17194.00,0,True,TRADEABLE,1


In [30]:
## Get info about a market
from autoIG.epics import US_CRUDE_OIL_EPIC,GOLD_EPIC
market = ig_service.fetch_market_by_epic(GOLD_EPIC)
print(f"Keys: {list(market.keys())}")

Keys: ['instrument', 'dealingRules', 'snapshot']


In [31]:
## Details about the market instrument
i,d,s = market_series(market)

## Get historical prices

In [8]:
result = ig_service.fetch_historical_prices_by_epic(epic = US_CRUDE_OIL_EPIC,resolution ='1Min',numpoints = 10 )
print(f"Keys: {list(result.keys())}")

Keys: ['prices', 'instrumentType', 'metadata']


In [9]:
# ask price is the lowest a seller will sell
# bid is highest a buyer will buy
# You buy at bid. sell at ask
result['prices'].head(2)

bid                             ask                  \
                       Open    High     Low   Close    Open    High     Low   
DateTime                                                                      
2022-12-06 20:14:00  7439.2  7440.2  7437.2  7438.7  7442.0  7443.3  7440.3   
2022-12-06 20:15:00  7438.5  7440.8  7437.5  7438.5  7442.3  7444.3  7441.0   

                            last                        
                      Close Open High Low Close Volume  
DateTime                                                
2022-12-06 20:14:00  7442.5  NaN  NaN NaN   NaN     44  
2022-12-06 20:15:00  7441.3  NaN  NaN NaN   NaN     78

In [10]:
result['instrumentType']

'COMMODITIES'

In [11]:
result['metadata']

{'allowance': {'remainingAllowance': 9867,
  'totalAllowance': 10000,
  'allowanceExpiry': 598702},
 'size': 10,
 'pageData': {'pageSize': 20, 'pageNumber': 1, 'totalPages': 1}}

In [25]:
## open a possition
from autoIG.epics import BITCOIN_EPIC,GOLD_EPIC,US_CRUDE_OIL_EPIC
resp = ig_service.create_open_position(
        currency_code='GBP',
        direction='BUY',
        epic=US_CRUDE_OIL_EPIC,
        order_type='MARKET',
        expiry='DFB',
        force_open='false',
        guaranteed_stop='false',
        size=1.0,
        level=None,
        limit_distance=None,
        limit_level=None,
        quote_id=None,
        stop_level=None,
        stop_distance=None,
        trailing_stop=None,
        trailing_stop_increment=None
        )
resp

{'date': '2022-12-08T13:53:20.108',
 'status': 'OPEN',
 'reason': 'SUCCESS',
 'dealStatus': 'ACCEPTED',
 'epic': 'CC.D.CL.USS.IP',
 'expiry': 'DFB',
 'dealReference': 'ZFPZ72ZMNSTZEB',
 'dealId': 'DIAAAALBUJ7XZA6',
 'affectedDeals': [{'dealId': 'DIAAAALBUJ7XZA6', 'status': 'OPENED'}],
 'level': 7483.6,
 'size': 1.0,
 'direction': 'BUY',
 'stopLevel': None,
 'limitLevel': None,
 'stopDistance': None,
 'limitDistance': None,
 'guaranteedStop': False,
 'trailingStop': False,
 'profit': None,
 'profitCurrency': None}

In [26]:
resp['dealStatus']

'ACCEPTED'

In [32]:
open_positions = ig_service.fetch_open_positions()
open_positions

,contractSize,createdDate,createdDateUTC,dealId,dealReference,size,direction,limitLevel,level,currency,...,percentageChange,netChange,bid,offer,updateTime,updateTimeUTC,delayTime,streamingPricesAvailable,marketStatus,scalingFactor
0,1.0,2022/12/09 08:27:03:000,2022-12-09T08:27:03,DIAAAALB3QYKFAZ,YNY27GSSWSWTZEB,1.0,BUY,None,7182.1,GBP,...,0.71,50.9,7167.6,7175.4,21:59:52,21:59:52,0,True,EDITS_ONLY,1
1,1.0,2022/12/09 08:31:25:000,2022-12-09T08:31:25,DIAAAALB3QDW6AJ,TAZC6K9DHLATZEB,1.0,BUY,None,7171.1,GBP,...,0.71,50.9,7167.6,7175.4,21:59:52,21:59:52,0,True,EDITS_ONLY,1
2,1.0,2022/12/09 08:31:29:000,2022-12-09T08:31:29,DIAAAALB3P482A5,C3LYQPCNVJJTZEB,1.0,BUY,None,7173.1,GBP,...,0.71,50.9,7167.6,7175.4,21:59:52,21:59:52,0,True,EDITS_ONLY,1
3,1.0,2022/12/09 08:32:35:000,2022-12-09T08:32:35,DIAAAALB3QZQLBB,SDL86LLAZCATZEB,1.0,BUY,None,7187.1,GBP,...,0.71,50.9,7167.6,7175.4,21:59:52,21:59:52,0,True,EDITS_ONLY,1
4,1.0,2022/12/09 08:32:36:000,2022-12-09T08:32:36,DIAAAALB3Q3H8A4,VA5JMWKGYDWTZEB,1.0,BUY,None,7187.1,GBP,...,0.71,50.9,7167.6,7175.4,21:59:52,21:59:52,0,True,EDITS_ONLY,1
5,1.0,2022/12/09 08:32:37:000,2022-12-09T08:32:37,DIAAAALB3RSBNA8,TUB7DVUGQX2TZEB,1.0,BUY,None,7186.1,GBP,...,0.71,50.9,7167.6,7175.4,21:59:52,21:59:52,0,True,EDITS_ONLY,1
6,1.0,2022/12/09 08:32:38:000,2022-12-09T08:32:38,DIAAAALB3Q9RRAX,95BPXGDKLZATZEB,1.0,BUY,None,7186.1,GBP,...,0.71,50.9,7167.6,7175.4,21:59:52,21:59:52,0,True,EDITS_ONLY,1
7,1.0,2022/12/09 08:32:38:000,2022-12-09T08:32:38,DIAAAALB3QNNBAS,R2SCQBFCLGETZEB,1.0,BUY,None,7186.1,GBP,...,0.71,50.9,7167.6,7175.4,21:59:52,21:59:52,0,True,EDITS_ONLY,1
8,1.0,2022/12/09 08:32:39:000,2022-12-09T08:32:39,DIAAAALB3Q7F2BD,EG7Z7WH42ANTZEB,1.0,BUY,None,7187.1,GBP,...,0.71,50.9,7167.6,7175.4,21:59:52,21:59:52,0,True,EDITS_ONLY,1
9,1.0,2022/12/09 08:32:40:000,2022-12-09T08:32:40,DIAAAALB3RKEDAL,PFSLH3R6PCNTZEB,1.0,BUY,None,7187.1,GBP,...,0.71,50.9,7167.6,7175.4,21:59:52,21:59:52,0,True,EDITS_ONLY,1


## Account activity

In [33]:
from_date = datetime(2021, 1, 1)
activities = ig_service.fetch_account_activity(from_date=from_date)
activities.head(3)

,date,epic,period,dealId,channel,type,status,description,details
0,2022-12-11T14:02:00,CS.D.BITCOIN.TODAY.IP,DFB,DIAAAALB5TRMZAP,PUBLIC_WEB_API,POSITION,REJECTED,Rejected: CLOSING ORDERS ONLY,None
1,2022-12-11T14:01:00,CS.D.BITCOIN.TODAY.IP,DFB,DIAAAALB5TVJ7A8,PUBLIC_WEB_API,POSITION,REJECTED,Rejected: CLOSING ORDERS ONLY,None
2,2022-12-11T14:00:01,CS.D.BITCOIN.TODAY.IP,DFB,DIAAAALB5TNTVAY,PUBLIC_WEB_API,POSITION,REJECTED,Rejected: CLOSING ORDERS ONLY,None


In [35]:
activities

,date,epic,period,dealId,channel,type,status,description,details
0,2022-12-11T14:02:00,CS.D.BITCOIN.TODAY.IP,DFB,DIAAAALB5TRMZAP,PUBLIC_WEB_API,POSITION,REJECTED,Rejected: CLOSING ORDERS ONLY,None
1,2022-12-11T14:01:00,CS.D.BITCOIN.TODAY.IP,DFB,DIAAAALB5TVJ7A8,PUBLIC_WEB_API,POSITION,REJECTED,Rejected: CLOSING ORDERS ONLY,None
2,2022-12-11T14:00:01,CS.D.BITCOIN.TODAY.IP,DFB,DIAAAALB5TNTVAY,PUBLIC_WEB_API,POSITION,REJECTED,Rejected: CLOSING ORDERS ONLY,None
3,2022-12-11T13:59:00,CS.D.BITCOIN.TODAY.IP,DFB,DIAAAALB5TE9EA3,PUBLIC_WEB_API,POSITION,REJECTED,Rejected: CLOSING ORDERS ONLY,None
4,2022-12-10T15:43:56,CC.D.CL.USS.IP,DFB,DIAAAALB5MVLKAZ,PUBLIC_WEB_API,POSITION,REJECTED,Rejected: Market closed,None
...,...,...,...,...,...,...,...,...,...
414,2021-10-25T13:46:55,UB.D.DRVNUS.DAILY.IP,DFB,DIAAAAGUFKR9DAV,WEB,WORKING_ORDER,REJECTED,Rejected: Order level too close to market level,None
415,2021-10-21T13:09:43,CS.D.GBPUSD.TODAY.IP,DFB,DIAAAAGTVTNWYA2,WEB,POSITION,ACCEPTED,Position/s closed: TVTLBXAJ,None
416,2021-10-21T13:08:46,CS.D.GBPUSD.TODAY.IP,DFB,DIAAAAGTVTLBXAJ,WEB,POSITION,ACCEPTED,Position opened: TVTLBXAJ,None
417,2021-10-15T13:37:21,CS.D.USDJPY.TODAY.IP,DFB,DIAAAAGR94EW6BD,WEB,POSITION,ACCEPTED,Position/s closed: R93HRKAV,None
